In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.


In [2]:
import glob, os
import imageio
import numpy as np
os.chdir(os.getcwd() + "/screenshots")
images = list()
for file in glob.glob("*.png"):
    images.append(imageio.imread(file))

arr = np.asarray(images)
os.chdir("..")

In [3]:
arr.shape

(699, 608, 1264, 3)

In [4]:
input_img = Input(shape=(608, 1264, 3))

In [5]:
## ENCODER
conv_1 = Conv2D(filters = 128, kernel_size = (16, 16), activation = "relu", padding = "same")(input_img)
encoded_1 = MaxPooling2D(pool_size = (2, 2), padding = "same")(conv_1)
conv_2 = Conv2D(filters = 64, kernel_size = (16, 16), activation = "relu", padding = "same")(encoded_1)
encoded_2 = MaxPooling2D(pool_size = (2, 2), padding = "same")(conv_2)

## DECODER
conv_3 = Conv2D(filters = 64, kernel_size = (16, 16), activation = "relu", padding = "same")(encoded_2)
upsample_1 = UpSampling2D(size = (2, 2))(conv_3)
conv_4 = Conv2D(filters = 128, kernel_size = (16, 16), activation = "relu", padding = "same")(upsample_1)
upsample_2 = UpSampling2D(size = (2, 2))(conv_4)
decoded = Conv2D(filters = 3, kernel_size = (16, 16), activation = "sigmoid", padding = "same")(upsample_2)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer = "adadelta", loss = "binary_crossentropy")

In [7]:
x_train = arr[50:75]
x_train = x_train / 255

In [8]:
x_train.shape

(25, 608, 1264, 3)

In [ ]:
encoder = Model(input_img, encoded_2)

In [ ]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 608, 1264, 3)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 608, 1264, 128)    98432     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 304, 632, 128)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 304, 632, 64)      2097216   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 152, 316, 64)      0         
Total params: 2,195,648
Trainable params: 2,195,648
Non-trainable params: 0
_________________________________________________________________


In [ ]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 608, 1264, 3)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 608, 1264, 128)    98432     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 304, 632, 128)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 304, 632, 64)      2097216   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 152, 316, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 152, 316, 64)      1048640   
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 304, 632, 64)      0         
__________

In [ ]:
autoencoder.fit(x_train, x_train, epochs = 5, batch_size = 5, validation_data = (x_train, x_train))

Instructions for updating:
Use tf.cast instead.
Train on 25 samples, validate on 25 samples
Epoch 1/5


In [ ]:
x_test = arr[400:405]

In [ ]:
x_test = x_test / 255

In [ ]:
decoded_imgs = autoencoder.predict(x_test)

In [ ]:
decoded_imgs[0].shape

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(decoded_imgs[4])
plt.show()

In [ ]:
decoded_imgs.shape

In [ ]:
plt.imshow(x_test[4])
plt.show()

In [ ]:
results = encoder.predict(x_test)

In [ ]:
results[4].shape

In [ ]:
plt.imshow(results[4].reshape(10*20, 64).T)
plt.show()